# Carregamento da base de dados

In [ ]:
import pandas as pd

In [ ]:
# Exibir dados
import plotly.express as px
import seaborn as sns
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
df = pd.read_csv(r'base_dados.csv', encoding='utf-8')
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
# Verifica o ID e os maiores valores encontrados em cada coluna
#pd.concat([df.idxmax(), df.max()], axis=1)

# Tratamento do atributo `Data`

In [ ]:
# Converte o atributo 'Data' para o tipo 'pandas.datetime'
df['Data'] = pd.to_datetime(df['Data'])
df.info()

# Criação dos atributos `Mês` e `Ano`

In [ ]:
# Criar colunas especificas para o Mês e o Ano do registro
df.insert(1, 'Mes', df['Data'].dt.month)
df.insert(2, 'Ano', df['Data'].dt.year)
df.head()

In [ ]:
df.info()

# Análise de Dados

## Analítica

### Vendas ao Longo do Tempo
Permite acompanhar a evolução das vendas ao longo do tempo, identificando tendências e padrões.

* **Relação**: Data e Hora x Valor Unitário
* **Gráfico**: Linhas ou Área

In [ ]:
compra_venda_tempo = df.groupby(['Ano', 'Tipo'], as_index=False)['Valor Unitário'].mean().round(2)
display(compra_venda_tempo)

In [ ]:
sns.lineplot(data=compra_venda_tempo, x='Ano', y='Valor Unitário', hue='Tipo', style='Tipo', markers=True);

In [ ]:
fig = px.line(compra_venda_tempo, x="Ano", y="Valor Unitário", color='Tipo', title='Variação Média de Preços ao longo dos Anos', markers=True)
fig.update_traces(textposition="top right")
fig.show()

In [ ]:
# Utilizando o Bokeh
plot = figure(width=900, height=600, title='Variação Média de Preços ao lonog dos Anos', toolbar_location=None)
line1 = compra_venda_tempo[compra_venda_tempo['Tipo'] == 'Compra']
line2 = compra_venda_tempo[compra_venda_tempo['Tipo'] == 'Venda']
plot.line(x=line1['Ano'], y=line1['Valor Unitário'], line_width=2, legend_label='Compra', color='green')
plot.line(x=line2['Ano'], y=line2['Valor Unitário'], line_width=2, legend_label='Venda', color='blue')
show(plot)

### Desempenho dos Produtos
Mostra quais produtos têm maior saida, auxiliando no planejamnto de estoque e promoções.
* **Relação**: Produto x Quantidade Vendida
* **Gráfico**: Barras

In [ ]:
desempenho_produto = df[df['Tipo'] == 'Venda'].groupby(['Produto'], as_index=False)['Quantidade'].sum().sort_values(by='Quantidade', ignore_index=True, ascending=False)
desempenho_produto_minimo = desempenho_produto['Quantidade'].min()*0.95
desempenho_produto_maximo = desempenho_produto['Quantidade'].max()*1.01

display(desempenho_produto)

In [ ]:
# Seaborn 1
gfg = sns.barplot(desempenho_produto, x="Produto", y="Quantidade")
gfg.set_ylim(desempenho_produto_minimo, desempenho_produto_maximo)
gfg.set_xticklabels(gfg.get_xticklabels(), rotation=90);

In [ ]:
# Seaborn 2
gfg = sns.barplot(desempenho_produto, x="Quantidade", y="Produto")
gfg.set_xlim(desempenho_produto_minimo, desempenho_produto_maximo)

In [ ]:
# Plotly Express 1
fig = px.bar(desempenho_produto, x="Produto", y="Quantidade", title='Desempenho dos Produtos')
fig.update_layout(yaxis_range=[desempenho_produto_minimo, desempenho_produto_maximo])
fig.show()

In [ ]:
# Plotly Express 2
fig = px.bar(desempenho_produto, x="Quantidade", y="Produto", title='Desempenho dos Produtos')
fig.update_layout(xaxis_range=[desempenho_produto_minimo, desempenho_produto_maximo], height=700)
fig.show()

In [ ]:
# Bokeh 1
bar_produtos = desempenho_produto['Produto'].values.tolist()
bar_quantidade = desempenho_produto['Quantidade'].values.tolist()

p = figure(x_range=bar_produtos, title='Desempenho dos Produtos', width=900, height=500)
p.vbar(x=bar_produtos, top=bar_quantidade, width=0.9)
p.xaxis.major_label_orientation = "vertical"
p.xaxis.subgroup_label_orientation = "normal"
p.y_range.start = desempenho_produto_minimo
p.y_range.end = desempenho_produto_maximo
show(p)

In [ ]:
# Bokeh 2
bar_produtos = desempenho_produto['Produto'].values.tolist()
bar_quantidade = desempenho_produto['Quantidade'].values.tolist()

p = figure(y_range=bar_produtos, title='Desempenho dos Produtos', width=600, height=600)
p.hbar(y=bar_produtos, right=bar_quantidade, height=0.9)
p.x_range.start = desempenho_produto_minimo
p.x_range.end = desempenho_produto_maximo
show(p)

### Lucratividade por Categoria
Avalia a margem de lucro por categoria, informando sobre a rentabilidade dos seguimentos.
* **Relação**: Categoria do Produto x Lucro (Quantidade do Produto x (Valor Unitário - Valor do Custo))
* **Gráfico**: Barras Empilhadas

In [ ]:
categoria_quantidade_produto_total = df.groupby(['Categoria'], as_index=False)['Quantidade'].sum()
categoria_quantidade_produto_total.sort_values(by='Quantidade', ascending=False, inplace=True)
categoria_quantidade_produto_total_max = int(categoria_quantidade_produto_total['Quantidade'].max() * 1.01)
categoria_quantidade_produto_total_min = int(categoria_quantidade_produto_total['Quantidade'].min()*0.95)
display(categoria_quantidade_produto_total)

In [ ]:
# Seaborn 1
gfg = sns.barplot(x=categoria_quantidade_produto_total['Categoria'], y=categoria_quantidade_produto_total['Quantidade'])
gfg.set_ylim(categoria_quantidade_produto_total_min, categoria_quantidade_produto_total_max)
gfg.set_xticklabels(gfg.get_xticklabels(), rotation=90);

In [ ]:
# Seaborn 2
gfg = sns.barplot(x=categoria_quantidade_produto_total['Quantidade'], y=categoria_quantidade_produto_total['Categoria'])
gfg.set_xlim(categoria_quantidade_produto_total_min, categoria_quantidade_produto_total_max);

In [ ]:
# Plotly Express 1
fig = px.bar(categoria_quantidade_produto_total, x="Categoria", y="Quantidade", title='Vendas por Categoria')
fig.update_layout(yaxis_range=[categoria_quantidade_produto_total_min, categoria_quantidade_produto_total_max])
fig.show()

In [ ]:
# Plotly Express 2
fig = px.bar(categoria_quantidade_produto_total, x="Quantidade", y="Categoria", title='Vendas por Categoria')
fig.update_layout(xaxis_range=[categoria_quantidade_produto_total_min, categoria_quantidade_produto_total_max])
fig.show()

////////////////////////////////////////////

In [ ]:
categoria_valor_unitario_total = df.groupby(['Categoria'], as_index=False)['Valor Unitário'].sum()
categoria_valor_unitario_total.sort_values(by='Valor Unitário', ascending=False, inplace=True)
categoria_valor_unitario_total_max = int(categoria_valor_unitario_total['Valor Unitário'].max() * 1.01)
categoria_valor_unitario_total_min = int(categoria_valor_unitario_total['Valor Unitário'].min()*0.95)
display(categoria_valor_unitario_total)

In [ ]:
# Seaborn 1
gfg = sns.barplot(x=categoria_valor_unitario_total['Categoria'], y=categoria_valor_unitario_total['Valor Unitário'])
gfg.set_xticklabels(gfg.get_xticklabels(), rotation=90);

In [ ]:
# Seaborn 2
gfg = sns.barplot(x=categoria_valor_unitario_total['Valor Unitário'], y=categoria_valor_unitario_total['Categoria']);

In [ ]:
# Plotly Express 1
fig = px.bar(categoria_valor_unitario_total, x="Categoria", y="Valor Unitário", title='Vendas por Categoria', color='Categoria')
fig.show()

In [ ]:
# Plotly Express 2
fig = px.bar(categoria_valor_unitario_total, x="Valor Unitário", y="Categoria", title='Vendas por Categoria', color='Categoria')
fig.show()

////////////////////////////////////////////

In [ ]:
categoria_custo_total = df.groupby(['Categoria'], as_index=False)['Custo'].sum()
categoria_custo_total.sort_values(by='Custo', ascending=False, inplace=True)
categoria_custo_total_max = int(categoria_custo_total['Custo'].max() * 1.01)
categoria_custo_total_min = int(categoria_custo_total['Custo'].min() * 0.95)
display(categoria_custo_total)

In [ ]:
# Seaborn 1
gfg = sns.barplot(x=categoria_custo_total['Categoria'], y=categoria_custo_total['Custo'])
gfg.set_xticklabels(gfg.get_xticklabels(), rotation=90);

In [ ]:
# Seaborn 1
gfg = sns.barplot(x=categoria_custo_total['Custo'], y=categoria_custo_total['Categoria']);

In [ ]:
# Plotly Express 1
fig = px.bar(categoria_custo_total, x="Categoria", y="Custo", title='Vendas por Categoria', color='Categoria')
fig.show()

In [ ]:
# Plotly Express 2
fig = px.bar(categoria_custo_total, x="Custo", y="Categoria", title='Vendas por Categoria', color='Categoria')
fig.show()

////////////////////////////////////////////

Problemas básicos de como estruturar esta seleção para fazer um bar-charts

In [ ]:
categoria_quantidade_custo_unitario_total = df.groupby(['Categoria'], as_index=False)[['Quantidade', 'Custo', 'Valor Unitário']].sum()
categoria_quantidade_custo_unitario_total.sort_values(by='Custo', ascending=False, inplace=True)
categoria_quantidade_custo_unitario_total_max = int(categoria_quantidade_custo_unitario_total['Custo'].max() * 1.01)
categoria_quantidade_custo_unitario_total_min = int(categoria_quantidade_custo_unitario_total['Custo'].min() * 0.95)
display(categoria_quantidade_custo_unitario_total)

In [ ]:
# Seaborn 1

# Seaborn 2

In [ ]:
# Plotly Express 1

# Plotly Express 2

////////////////////////////////////////////

In [ ]:
lucratividade_categoria = pd.DataFrame()
lucratividade_categoria['Categoria'] = df_locratividade_categoria['Categoria']
lucratividade_categoria['Lucro'] = df_locratividade_categoria['Valor Unitário'] - df_locratividade_categoria['Custo']

lucratividade_categoria.sort_values(by='Lucro', ascending=False, inplace=True)

display(lucratividade_categoria)

In [ ]:
# Plotly Express 1
fig = px.bar(lucratividade_categoria, x="Categoria", y="Lucro", title='Lucro por Categoria', color='Categoria')
fig.show()

### Fluxo de Entrada e Saída
Monitora o fluxo de produtos, ajudando a entender o movimento do inventário.
* **Relação**: Data x Entrada ou Saida de Produto (Quantidade) (Compra x Venda)
* **Gráfico**: Fluxo de Caixa

### Canais de Venda
Compara a performance de vendas online versus loja física, orientando estratégias de marketing e venda.
* **Relação**: Meio de Venda (Canal de Venda) x Valor Total de Vendas.
* **Pizza** ou Donut.

In [ ]:
canal_venda_valor_unitario = df.groupby('Canal de Venda')['Valor Unitário'].sum()
display(canal_venda_valor_unitario)

In [ ]:
fig = px.pie(canal_venda_valor_unitario, values=canal_venda_valor_unitario.values, names=canal_venda_valor_unitario.index, title='Relação entre Meio de Cenda e Total de Vendas')
fig.update_traces(textposition='inside', textinfo='percent+label', textfont_size=20)
fig.update_layout(margin=dict(t=70, b=20, l=0, r=0), width=600)
fig.show();

### Preferência de Pagamento
Identifica as formas de pagamento preferidas pelos clientes, podendo influenciar as opções oferecidas.
* **Relação**: Forma de Pagamento (Método de Pagamento) x Valor Total
* **Gráfico**: Barras Horizontais

In [ ]:
metodo_pagamento_valor_total = df.groupby('Método de Pagamento')['Valor Unitário'].sum()
display(metodo_pagamento_valor_total)

In [ ]:
# Plotly Express 1
fig = px.bar(metodo_pagamento_valor_total, x=metodo_pagamento_valor_total.index, y=metodo_pagamento_valor_total.values, title='Preferência de Pagamento', color=metodo_pagamento_valor_total.index)
fig.show()

In [ ]:
# Plotly Express 2
fig = px.bar(metodo_pagamento_valor_total, x=metodo_pagamento_valor_total.values, y=metodo_pagamento_valor_total.index, title='Preferência de Pagamento', color=metodo_pagamento_valor_total.index)
fig.show()

In [ ]:
fig = px.pie(metodo_pagamento_valor_total, values=metodo_pagamento_valor_total.values, names=metodo_pagamento_valor_total.index, title='Preferência de Pagamento')
fig.update_traces(textposition='inside', textinfo='percent+label', textfont_size=20)
fig.update_layout(margin=dict(t=70, b=20, l=0, r=0), width=600)
fig.show();

### Análise Geográfica
Destaca as áreas geográficas com maior volume de vendas, direcionando esforços de marketing localizados.
* **Relação**: Cidade e Estado x Valor Total de Venda
* **Gráfico**: Mapa de Calor

# Para um outro dia...

### Relacionamento de Dados

In [ ]:
# Relacionar Data de Transição com o volume de compra e venda para identificar padrões ao longo do tempo
vol_compra_venda = df.groupby(['Ano', 'Mes','Tipo'], as_index=False)['Quantidade'].sum()
display(vol_compra_venda)

### Análise de Produtos

In [ ]:
# Correlacionar o Produto com a quantidade vendida (produtos mais populares)
vol_produto_vendido = df[(df['Tipo'] == 'Venda')].groupby(['Ano', 'Produto'], as_index=False)['Quantidade'].sum()
display(vol_produto_vendido)

In [ ]:
display(vol_produto_vendido)

In [ ]:
# Os 3 (três) produtos mais vendidos entre 2010 e 2024
vol_produto_vendido.iloc[vol_produto_vendido['Quantidade'].argsort()[-3:]]

In [ ]:
vol_produto_vendido.groupby(['Ano'], as_index=False)['Quantidade'].sum()

### Análise de Preço

In [ ]:
# Associar valor unitário do produto com o tipo de transação para entender a dinâmica de preços

## Tática

## Estratégica

# Exportação da Base de Dados

In [ ]:
display(df)

In [ ]:
df.to_csv(r'base_dados_tratada.csv', index=False, encoding='utf-8')